In [1]:
import os, sys
global top_path  # the path of the top_level directory
global script_path, data_path, logging_path
top_path = '/scratch/user/zshuying/ppi_mutation/'
sys.path.append(top_path)
pj=os.path.join
script_path, data_path, logging_path= os.path.join(top_path,'scripts'),\
    os.path.join(top_path,'data'),\
    os.path.join(top_path,'logs')
from scripts.baseline1.datasets_baseline1 import *


/scratch/user/zshuying/.conda/envs/gnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_sparse import SparseTensor
loader = variantPPI.train_dataloader()
for batch in loader:
    adj = SparseTensor(row=batch.edge_index[0], col=batch.edge_index[1])
    break

In [2]:
variantPPI=VariantPPIModule(root=pj(data_path,'baseline1'),clinvar_csv=pj(data_path,'clinvar','mutant_seq_2019_1_no_error.csv'),variant_embedding_path=pj(data_path,'baseline0','2019_variant_embds.pt'),
                            wild_embedding_path=pj(data_path,'baseline0','all_wild_embeddings.pt'),batch_size=20,num_workers=15,random_seed=1050,train_val_ratio=0.8)
# loader = variantPPI.train_dataloader()esxdhgfdesxdfxsds5xsdeww3dedrdffed des34rtedc
# loader=VariantRandomWalkSamler(variantPPI.trainset,walk_length=3,shuffle=True,batch_size=variantPPI.batch_size,num_workers=variantPPI.num_workers)
# for batch in loader:
#     batch
#     break



Split dataset into train, val with the rate of 0.8


In [30]:
CONFIG_PATH='/scratch/user/zshuying/ppi_mutation/configs'
import yaml

def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)

    return config


config = load_config('b1_test.yaml')


In [31]:
variantPPI=VariantPPIModule(**config['data_init'])

Split dataset into train, val with the rate of 0.8


In [8]:
variantPPI.dataset.num_nodes*20

223720

In [7]:
batch.x[0].shape,batch.x[1].shape,batch.x[2].shape

(torch.Size([223720, 5760]), torch.Size([115200]), torch.Size([20]))

In [9]:
batch

DataBatch(x=[3], edge_index=[2, 5771640], y=[1], num_nodes=223720, batch=[223720], ptr=[21])

In [10]:
from scripts.baseline1.baseline1_models import *


In [ ]:
gnn_test=gnn()

In [24]:
batch.batch

tensor([0, 0, 0,  ..., 1, 1, 1])

In [10]:
from torch_geometric.data.lightning import LightningDataset

In [11]:
v=LightningDataset(split_train_val(variantPPI.dataset,train_val_split=0.8,random_seed=5))

Split dataset into train, val with the rate of 0.8


In [16]:
for batch in v.train_dataloader():
    batch

TypeError: DataLoader found invalid type: <class 'torch.utils.data.dataset.Subset'>

In [6]:
loader = variantPPI.train_dataloader()
for batch in loader:
    batch
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 150, in collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeErr

In [7]:
%debug

> /scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/_utils.py(644)reraise()
    642             # instantiate since we don't know how to
    643             raise RuntimeError(msg) from None
--> 644         raise exception
    645 
    646 

> /scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/dataloader.py(1371)_process_data()
   1369         self._try_put_index()
   1370         if isinstance(data, ExceptionWrapper):
-> 1371             data.reraise()
   1372         return data
   1373 

> /scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/dataloader.py(1345)_next_data()
   1343             else:
   1344                 del self._task_info[idx]
-> 1345                 return self._process_data(data)
   1346 
   1347     def _try_put_index(self):

> /scratch/user/zshuying/.conda/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/dataloader.py(633)__next__()
    631         

In [25]:
variantPPI.dataset.process()


data_2347.pt saved
data_2348.pt saved
data_2349.pt saved
data_2350.pt saved
data_2351.pt saved
data_2352.pt saved
data_2353.pt saved
data_2354.pt saved
data_2355.pt saved
data_2356.pt saved
data_2357.pt saved
data_2358.pt saved
data_2359.pt saved
data_2360.pt saved
data_2361.pt saved
data_2362.pt saved
data_2363.pt saved
data_2364.pt saved
data_2365.pt saved
data_2366.pt saved
data_2367.pt saved
data_2368.pt saved
data_2369.pt saved
data_2370.pt saved
data_2371.pt saved
data_2372.pt saved
data_2373.pt saved
data_2374.pt saved
data_2375.pt saved
data_2376.pt saved
data_2377.pt saved
data_2378.pt saved
data_2379.pt saved
data_2380.pt saved
data_2381.pt saved
data_2382.pt saved
data_2383.pt saved
data_2384.pt saved
data_2385.pt saved
data_2386.pt saved
data_2387.pt saved
data_2388.pt saved
data_2389.pt saved
data_2390.pt saved
data_2391.pt saved
data_2392.pt saved


In [25]:
clinvar=pd.read_csv(pj(data_path,'clinvar','mutant_seq_2019_1_no_error.csv'))
clinvar[clinvar['Name']=='NM_000162.3(GCK):c.1157T>C (p.Leu386Pro)']

,Unnamed: 0.1,Unnamed: 0,#AlleleID,Type,Name,Label,ClinicalSignificance,ClinSigSimple,ReviewStatus,OtherIDs,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,UniProt,Seq
1789,1879,42316,44844,single nucleotide variant,NM_000162.3(GCK):c.1157T>C (p.Leu386Pro),1,Likely pathogenic,1,"criteria provided, multiple submitters, no con...",-,"Dec 22, 2016",193922268,-,RCV000029843;RCV000517698,P35557,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...


In [4]:
dic_list=[]
from collections import ChainMap
for i in range(10):
    dic_list.append(torch.load('/scratch/user/zshuying/ppi_mutation/data/baseline0/all_wild_embeddings_%s.pt'%i))
final_dic=dict(ChainMap(*dic_list))
torch.save(final_dic,'/scratch/user/zshuying/ppi_mutation/data/baseline0/all_wild_embeddings.pt')

In [10]:
exclude_self=[item for item in interactions if '-' in item]

In [11]:
interactions_eges=[(item.split('-')[0],item.split('-')[1]) for item in exclude_self]

In [12]:
interactions_eges

[('O43741', 'Q9P0T4'),
 ('Q9Y2R9', 'O60783'),
 ('O14958', 'Q9ULJ3'),
 ('Q96GF1', 'Q8NBI2'),
 ('Q13136', 'Q8NHQ1'),
 ('Q08117', 'Q9NWQ4'),
 ('O14879', 'Q96CS2'),
 ('P59817', 'Q86YH2'),
 ('Q9BYU1', 'Q5MJ09'),
 ('Q96LB9', 'Q9UHD4'),
 ('Q6UWI2', 'Q9UHD9'),
 ('Q8IUQ4', 'Q9UPY8'),
 ('Q7Z6G3', 'Q9NP79'),
 ('Q9UMS4', 'Q13435'),
 ('P31937', 'P21964'),
 ('Q93062', 'Q92567'),
 ('Q9NP66', 'Q96BS2'),
 ('Q12905', 'Q08211'),
 ('Q9H9Y6', 'O15160'),
 ('P18031', 'Q9NPL8'),
 ('O43561', 'Q8NBJ4'),
 ('Q9NPQ8', 'Q9Y371'),
 ('Q9UPZ6', 'Q9NY72'),
 ('Q8N1B4', 'Q6NYC8'),
 ('Q13630', 'P49903'),
 ('Q9BQY4', 'Q3LI64'),
 ('P61970', 'Q8IY57'),
 ('Q99459', 'Q08AF8'),
 ('Q86U28', 'Q9H5Z6'),
 ('Q92917', 'P31321'),
 ('P34810', 'Q96CS7'),
 ('O43395', 'Q9Y2I6'),
 ('P54105', 'P62308'),
 ('Q9UI95', 'Q9C0B1'),
 ('P86478', 'Q03989'),
 ('A8MQ03', 'Q9NVF9'),
 ('Q08379', 'Q96C55'),
 ('P55209', 'P19338'),
 ('Q9H2F3', 'Q9Y320'),
 ('Q93062', 'Q9BVN2'),
 ('Q9NW97', 'Q9H6H4'),
 ('Q7Z6R9', 'P0C7H8'),
 ('P83876', 'Q96D03'),
 ('Q17RD7',

In [12]:
import networkx as nx
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
G=nx.Graph(name='Protein Interaction Graph')


In [13]:
G.add_edges_from(interactions_eges)

In [14]:
print(G)

Graph named 'Protein Interaction Graph' with 11340 nodes and 72955 edges


In [15]:
print('Average degree', sum(dict(G.degree).values()) / len(G.nodes))


Average degree 12.866843033509701


In [16]:
12**4

20736

In [20]:
def merge_values_from_keys(dictionary, keys):
    merged_values = set()

    for key in keys:
        if key in dictionary:
            merged_values.update(dictionary[key])

    return merged_values
k_hop=pd.DataFrame(columns=['1-hop neighbors','2-hop neighbors','3-hop neighbors','4-hop neighbors','5-hop neighbors','6-hop neighbors'])
for n, nbrs in G.adj.items():
    k_hop.loc[n,'1-hop neighbors']= len(nbrs)
    hop2=merge_values_from_keys(G.adj,set(nbrs.keys()))
    hop12=hop2.union(set(nbrs.keys()))
    k_hop.loc[n,'2-hop neighbors']= len(hop12)
    hop3=merge_values_from_keys(G.adj,hop2)
    hop123=hop3.union(hop12)
    k_hop.loc[n,'3-hop neighbors']= len(hop123)
    hop4=merge_values_from_keys(G.adj,hop3)
    hop1234=hop4.union(hop123)
    k_hop.loc[n,'4-hop neighbors']= len(hop1234)  
    hop5=merge_values_from_keys(G.adj,hop4)
    hop12345=hop4.union(hop1234)
    k_hop.loc[n,'5-hop neighbors']= len(hop12345) 
    hop6=merge_values_from_keys(G.adj,hop5)
    hop123456=hop6.union(hop12345)
    k_hop.loc[n,'6-hop neighbors']= len(hop123456) 

In [63]:
k_hop

,1-hop neighbors,2-hop neighbors,3-hop neibors,4-hop neighbors,3-hop neighbors
O43741,144,3369,NaN,10575,8956.0


discard interactions which contain items that do not have sequences available

In [17]:
uniprots=pd.read_csv('/scratch/user/zshuying/ppi_mutation/ppi_seq_huri_humap.csv')['UniProt'].tolist()
with open('/scratch/user/zshuying/ppi_mutation/ppi_huri_humap.txt','r') as f:
    interactions=list(eval(f.readline()))

In [19]:
useful_interactions=[item for item in interactions if (item.split('-')[0] in uniprots and item.split('-')[1] in uniprots)]


In [20]:
with open('/scratch/user/zshuying/ppi_mutation/ppi_huri_humap.txt','w') as f:
    f.writelines(str(useful_interactions))

In [13]:
uniprots

0        Q9BQA9
1        Q8TAK5
2        P12956
3        P03999
4        O75787
          ...  
11181    O43581
11182    P48506
11183    P35244
11184    P63252
11185    Q13308
Name: UniProt, Length: 11186, dtype: object

In [16]:
'Q9BQA9' in uniprots.tolist()

True